In [4]:
import numpy as np
import os

## Carregando o Dataset

In [6]:
caminhos = [os.path.join("src", nome) for nome in os.listdir("src")]
print(caminhos)

['src/4_EMG_Claudia_25-11-2019_203111.npy', 'src/1_EMG_Claudia_25-11-2019_202048.npy', 'src/5_EMG_Claudia_25-11-2019_203540.npy', 'src/1_EMG_Claudia_25-11-2019_202406.npy', 'src/4_EMG_Claudia_25-11-2019_203206.npy', 'src/2_EMG_Claudia_25-11-2019_202532.npy', 'src/3_EMG_Claudia_25-11-2019_202949.npy']


In [9]:
dataset = [np.load(gesture) for gesture in caminhos]
dataset = np.array(dataset)
print(dataset.shape)

(7, 4, 1400, 4)
